The system
=======
A system consists mainly of three parts:
    - Entities. May represent agents or inactive entities like physical space.
    - Links. Represent connections between the entities. 
    - Time. The passage of time happens.
    
Basically a system is one big graph that passes information through the links between the nodes.

In [10]:
class System:
    """
    The global system and event dispatcher.

    Aware of hosts, space, and the passage of time (steps). It is
    the main struture of the program, being the only observable and given it
    coordinates all processes.

    """

    def __init__(self):
        'Initialize an empty system.'
        self.entities = set()
        self.entitynames = {}
        self.links = {}
        self.time = None
    # ---

    def addEntity(self, entity, name):
        'Add an entity to the graph.'
        self.entities.add(entity)
        self.entitynames[name] = entity
    # ---
        
    def step(self):
        'Take a single step forward in time.'
        # Process linked items
        #TODO...
        # Process each entity
        for entity in self.entities:
            entity.process(self.time)
    # ---
        
        
    def run(self, steps):
        'Start running the simulation.'
        # Init time
        if self.time is None:
            self.time = 0
        # Take steps
        for _ in range(steps):
            self.step()
    # ---
    
    def stateof(self, entityname):
        "Ask the entity for it's state"
        entity = self.entitynames[entityname]
        return entity.logstate()
    # ---
                
                

Entities
=====
An entity is something that resides in the system. 

It processes information according to it's internal state and the information flowing through the links it shares with other entities.

In [15]:
class Entity:
    '''An entity is something that resides in the system.

    It processes information according to it's internal state 
    and the information flowing through the links it shares with 
    other entities.
    
    An entity registers the following methods:
        - process(time)
        - logstate()
    '''
    
    def process(self, time):
        pass
    
    def logstate(self):
        pass
    
# --- Entity

In [14]:
# < --- Expected use case:

class Cells(Entity):
    'A special type of group entity that follows a Fibbonacci pattern'
    
    def __init__(self):
        self.current = 1
        self.previous = 0
        self.changes = {}
    
    def process(self, time):
        c = self.current
        p = self.previous
        self.current, self.previous = c+p, c
        print('Processed: ', self.logstate())
        
    def logstate(self):
        return f'Current cells: {self.current}'
# --- Cells


# Create a system
sys = System()

# Add the cells
sys.addEntity(Cells(), 
              name='cells')

# Run the processes in the system
sys.run(steps=10)

# Get the state of the cells
sys.stateof('cells')

Processed:  Current cells: 1
Processed:  Current cells: 2
Processed:  Current cells: 3
Processed:  Current cells: 5
Processed:  Current cells: 8
Processed:  Current cells: 13
Processed:  Current cells: 21
Processed:  Current cells: 34
Processed:  Current cells: 55
Processed:  Current cells: 89


'Current cells: 89'

Interactions
=======

Multiple entities can interact between them through Interactions.

Interactions link 2 or more entities to perform a series of actions together.

In [17]:
class Interaction:
    'A structure representing flow of information btw entities.'
    
    def __init__(self, entities, effect):
        self.entities = entities
        self.effects = [effect]
        
    def append(self, effect):
        'Effects btw the same entities can be appended and executed in order'
        self.effects.append(effect)
# --- Interaction    

In order to achieve this, the system has to have aditional control for the propagation of the information.

In [27]:
class System:
    """
    The global system and event dispatcher.

    Aware of hosts, space, and the passage of time (steps). It is
    the main struture of the program, being the only observable and given it
    coordinates all processes.

    """

    def __init__(self):
        'Initialize an empty system.'
        self.entities = set()
        self.toentity = {}
        self.toentityname = {}
        
        self.interactions = set()
        self.time = None
    # ---

    def addEntity(self, entity, name):
        'Add an entity to the graph.'
        self.entities.add(entity)
        self.toentity[name] = entity
        self.toentityname[entity] = name
    # ---
        
    def step(self):
        'Take a single step forward in time.'
        # Process linked items
        for inter in self.interactions:
            entities = inter.entities
            for effect in inter.effects:
                effect(*entities)
        # Process each entity
        for entity in self.entities:
            entity.process(self.time)
    # ---
         
    def run(self, steps):
        'Start running the simulation.'
        # Init time
        if self.time is None:
            self.time = 0
        # Take steps
        for _ in range(steps):
            self.step()
    # ---
    
    def stateof(self, entityname):
        "Ask the entity for it's state"
        entity = self.toentity[entityname]
        return entity.logstate()
    # ---
    
    def link(self, effect, entitynames):
        'Add a link between two named entities'
        entities = tuple(self.toentity[ename] 
                                for ename in entitynames)
        
        # Check if there is already some link btw
        # those same entities.
        if entities in self.interactions:
            # Add the new effect
            interactions[entities].append(effect)
            
        else:
            # Else, add the new link
            interaction = Interaction(entities, effect)
            self.interactions.add( interaction )
    # ---
# --- System
                

Below we show a simple use case where interactions play an important role.

In [64]:
# < --- Intended use case:

class Animals(Entity):
    'A simple group entity.'
    
    def __init__(self, pop=10):
        self.hunt_results = None
        self.population = pop
    # ---
        
    @property
    def population(self):
        return self._population
    # ---
    
    @population.setter
    def population(self, value):
        'The population consists of integers >= 0'
        if value < 0:
            self._population = 0
        else:
            self._population = int(value)
    # ---
                                   
    def logstate(self):
        return self.population
    # ---
# ---

def limit(quantity, interval):
    bottom, top = interval
    if quantity > top:
        quantity = top
        
    elif quantity < bottom:
        quantity = bottom
        
    return quantity
# ---
        
    
class Predators(Animals):
    def process(self, time):
        self.population += self.hunt_results
        self.hunt_results = None
        self.die()
        self.logstate()
        
    def die(self):
        self.population -= 0.3*self.population
        
    def logstate(self):
        print('Total predators: ', self.population)
# ---

class Preys(Animals):
    def process(self, time):
        # Reduce population due to hunting
        self.population -= self.hunt_results
        self.hunt_results = None
        self.reproduce()
        self.logstate()
        
    def logstate(self):
        print('Total preys: ', self.population)
        
    def reproduce(self):
        self.population *= 2
# ---
            

def predation(predators, preys):
    '''The predation interaction.
    
    h (hunting) is the effect of predators in prey.
    f (feeding) is the effect of prey in predators.
    '''
    h, f = 0.005, 0.0002
    close_encounters = preys.population * predators.population
    
    preys.hunt_results = h * close_encounters
    predators.hunt_results = limit(f * close_encounters,
                                   [0, predators.population])
    
    print(f'Predators are hunting {int(preys.hunt_results)} preys')
# ---
    

# Create the system
sys = System()

# Create the cells
sys.addEntity(Predators(pop=100),
              name='Predators')
sys.addEntity(Preys(pop=100),
              name='Preys')

# Add the link btw the cells
sys.link(predation, 
         entitynames=['Predators', 'Preys'])

# Verify the initial state
sys.stateof('Predators')
sys.stateof('Preys')

# Run the simulation
sys.run(steps=100)

Total predators:  100
Total preys:  100
Predators are hunting 50 preys
Total preys:  100
Total predators:  71
Predators are hunting 35 preys
Total preys:  128
Total predators:  50
Predators are hunting 32 preys
Total preys:  192
Total predators:  35
Predators are hunting 33 preys
Total preys:  316
Total predators:  25
Predators are hunting 39 preys
Total preys:  552
Total predators:  18
Predators are hunting 49 preys
Total preys:  1004
Total predators:  13
Predators are hunting 65 preys
Total preys:  1876
Total predators:  10
Predators are hunting 93 preys
Total preys:  3564
Total predators:  9
Predators are hunting 160 preys
Total preys:  6806
Total predators:  10
Predators are hunting 340 preys
Total preys:  12930
Total predators:  14
Predators are hunting 905 preys
Total preys:  24048
Total predators:  19
Predators are hunting 2284 preys
Total preys:  43526
Total predators:  26
Predators are hunting 5658 preys
Total preys:  75734
Total predators:  36
Predators are hunting 13632 prey

The use case we are mainly interested in is in simulating several cell types along with cellular automata fields for chemicals, so we proceed to move forwards towards that intended use case

Simple Cellular Automaton
===============

Now we tackle the problem of a simple 1D CA within our framework.

In [77]:
class CellularAutomaton(Entity):
    'A simple cellular automaton.'
    
    def __init__(self, n=25):
        # Place a single active cell in the middle
        self.gridstate = tuple(1 if i == n//2 else 0
                                   for i in range(n))
        self.rule = [0, 1, 0, 1, 1, 0, 1, 0]
        
    def process(self, time):
        self.gridstate = self.applyrule(self.gridstate, self.rule)
        print(''.join(self.logstate()))
        
    def applyrule(self, grid, rule):
        newgrid = list(grid)
        for i,cell in enumerate(grid):
            # Cells at the edges are constant
            if 1 <= i < len(grid)-1:
                neighbors = grid[i-1:i+2]
                newgrid[i] = self.rule[self.neighbor_n(neighbors)]
        return tuple(newgrid)
                
    def neighbor_n(self, neighbors):
        a, b, c = neighbors
        return c + b*2 + a*4
    
    def logstate(self):
        return [ '#' if cell else '.' for cell in self.gridstate ]
    
sys = System()

sys.addEntity(CellularAutomaton(80),
              name='Sierpinsky')

print( ''.join(sys.stateof('Sierpinsky')) )

sys.run(steps=100)

........................................#.......................................
.......................................#.#......................................
......................................#...#.....................................
.....................................#.#.#.#....................................
....................................#.......#...................................
...................................#.#.....#.#..................................
..................................#...#...#...#.................................
.................................#.#.#.#.#.#.#.#................................
................................#...............#...............................
...............................#.#.............#.#..............................
..............................#...#...........#...#.............................
.............................#.#.#.#.........#.#.#.#............................
............................

We note now, that it is important to be available to get a representation (as __repr__ and __str__) of the system itself, as for the entities and interactions, and not to lose the ability to get a data structure for the state. So, we make further changes to the system.

In [141]:
class Entity:
    '''An entity is something that resides in the system.

    It processes information according to it's internal state 
    and the information flowing through the links it shares with 
    other entities.
    
    An entity registers the following methods:
        - process(time)
        - logstate()
    '''
    def __init__(self):
        self.state = None
    
    def process(self, time):
        pass
# --- Entity



class System:
    """
    The global system and event dispatcher.

    Aware of hosts, space, and the passage of time (steps). It is
    the main struture of the program, being the only observable and given it
    coordinates all processes.

    """

    def __init__(self):
        'Initialize an empty system.'
        self.entities = set()
        self.toentity = {}
        self.toentityname = {}
        
        self.interactions = set()
        self.time = None
    # ---
    
    def __getitem__(self, entityname):
        'Access the entities by name.'
        return self.toentity[entityname]
    # ---

    def add(self, entity, name):
        'Add an entity to the graph.'
        self.entities.add(entity)
        self.toentity[name] = entity
        self.toentityname[entity] = name
    # ---
        
    def step(self):
        'Take a single step forward in time.'
        # Process linked items
        for inter in self.interactions:
            entities = inter.entities
            for effect in inter.effects:
                effect(*entities)
        # Process each entity
        for entity in self.entities:
            entity.process(self.time)
    # ---
    
        
    def start(self):
        'Initialize things to be initialized.'
        # Init time
        if self.time is None:
            self.time = 0
    # ---
         
    def run(self, steps):
        'Start running the simulation.'
        # Make sure things are initialized 
        self.start()
        
        # Take steps
        for _ in range(steps):
            self.step()
    # ---
    
    def stateof(self, entityname):
        "Ask the entity for it's state"
        return self[entityname].state
    # ---
    
    def link(self, effect, entitynames):
        'Add a link between named entities.'
        entities = tuple(self.toentity[ename] 
                                for ename in entitynames)
        
        # Check if there is already some link btw
        # those same entities.
        if entities in self.interactions:
            # Add the new effect
            interactions[entities].append(effect)
            
        else:
            # Else, add the new link
            interaction = Interaction(entities, effect)
            self.interactions.add( interaction )
    # ---
# --- System

In [86]:
# Intended use case...
class CellularAutomaton(Entity):
    'A simple cellular automaton.'
    
    def __init__(self, n=25):
        # Place a single active cell in the middle
        self.gridstate = tuple(1 if i == n//2 else 0
                                   for i in range(n))
        self.rule = [0, 1, 0, 1, 1, 0, 1, 0]
        
    def process(self, time):
        self.gridstate = self.applyrule(self.gridstate, self.rule)
        print(self)
        
    def applyrule(self, grid, rule):
        newgrid = list(grid)
        for i,cell in enumerate(grid):
            # Cells at the edges are constant
            if 1 <= i < len(grid)-1:
                neighbors = grid[i-1:i+2]
                newgrid[i] = self.rule[self.neighbor_n(neighbors)]
        return tuple(newgrid)
                
    def neighbor_n(self, neighbors):
        a, b, c = neighbors
        return c + b*2 + a*4
    
    @property
    def state(self):
        return [ '#' if cell else '.' for cell in self.gridstate ]
    
    def __str__(self):
        return ''.join(self.state)
# --- CellularAutomaton
    
sys = System()

sys.add(CellularAutomaton(80),
              name='Sierpinsky')

print(sys['Sierpinsky'] )

sys.run(steps=100)

........................................#.......................................
.......................................#.#......................................
......................................#...#.....................................
.....................................#.#.#.#....................................
....................................#.......#...................................
...................................#.#.....#.#..................................
..................................#...#...#...#.................................
.................................#.#.#.#.#.#.#.#................................
................................#...............#...............................
...............................#.#.............#.#..............................
..............................#...#...........#...#.............................
.............................#.#.#.#.........#.#.#.#............................
............................

We made an important change to the Entity object interface, as it can be seen, the method `logstate` was eliminated, and now the Entity state can and should be accessed with the `state` attribute. From this, now 
```python
    sys.stateof('SomeEntity') == sys['SomeEntity'].state == sys.toentity['SomeEntity'].state
```

Now, it is desirable to have the ability to modify the run behavior on the fly, so, one option is to allow for hooks in the run function and on adding the automaton.

In [136]:
import collections

class Interaction:
    'A structure representing flow of information btw entities.'
    
    def __init__(self, entities, effect):
        self.entities = entities
        self.effects = [effect]
        
    def append(self, effect):
        'Effects btw the same entities can be appended and executed in order'
        self.effects.append(effect)
        
    def process(self):
        'Executes the interaction.'
        for effect in self.effects:
            effect(*self.entities)
# --- Interaction 



class System:
    """
    The global system and event dispatcher.

    Aware of hosts, space, and the passage of time (steps). It is
    the main struture of the program, being the only observable and given it
    coordinates all processes.

    """

    def __init__(self):
        'Initialize an empty system.'
        self.entities = set()
        self.toentity = {}
        self.toentityname = {}
        
        self.interactions = {}
        self.time = None
        
        self.inithooks = {}
        self.prehooks = {}
        self.hooks = {}
    # ---
    
    def __getitem__(self, entityname):
        'Access the entities by name.'
        return self.toentity[entityname]
    # ---

    def add(self, entity, name, inithook=None):
        'Add an entity to the graph.'
        self.entities.add(entity)
        self.toentity[name] = entity
        self.toentityname[entity] = name
        
        if inithook:
            self.add_interaction_to(self.inithooks,
                                    inithook, 
                                    [name])
    # ---
    
    def process_interactions_in(self, interactions):
        'From the dict-like container of interactions, process items.'
        for interaction in interactions.values():
            interaction.process()
    # ---
        
    def step(self):
        'Take a single step forward in time.'
        # Process pre-step hooks
        self.process_interactions_in(self.prehooks)
        
        # Process linked items
        self.process_interactions_in(self.interactions)
        # Process each entity
        for entity in self.entities:
            entity.process(self.time)
            
        # Process post-step hooks
        self.process_interactions_in(self.hooks)
    # ---
    
        
    def start(self):
        'Initialize things before starting simulation.'
        # Init time
        if self.time is None:
            self.time = 0
        # Make the initialization actions
        self.process_interactions_in(self.inithooks)
    # ---
    
    def update_hooks(self, hooktype, newhooks):
        'Add the given hooks to the system.'
        if newhooks:
            for hook, entitynames in newhooks.items():
                self.add_interaction_to(hooktype, hook, entitynames)
    # ---
         
    def run(self, steps, inithooks=None, hooks=None, prehooks=None):
        'Start running the simulation.'
        # Handle hooks
        self.update_hooks(self.inithooks, inithooks)
        self.update_hooks(self.hooks, hooks)
        self.update_hooks(self.prehooks, prehooks)
        
        # Make sure things are initialized 
        self.start()
        
        # Take steps
        for _ in range(steps):
            self.step() 
    # ---
    
    def stateof(self, entityname):
        "Ask the entity for it's state"
        return self[entityname].state
    # ---
    
    def link(self, effect, entitynames):
        'Add an interation btw named entities.'
        self.add_interaction_to(self.interactions,
                                effect, 
                                entitynames)
    # ---
    
    def add_interaction_to(self, container, effect, entitynames):
        'Add an interaction btw named entities to a dict-like container.'
        entities = tuple(self.toentity[ename] 
                                for ename in entitynames)
        
        # Check if there is already some link btw
        # those same entities.
        if entities in container:
            # Add the new effect
            container[entities].append(effect)
            
        else:
            # Else, add the new link
            interaction = Interaction(entities, effect)
            container[entities] = interaction
    # ---
# --- System

In [ ]:
# Intended use case...
class CellularAutomaton(Entity):
    'A simple cellular automaton.'
    
    def __init__(self, n=25):
        # Place a single active cell in the middle
        self.gridstate = tuple(1 if i == n//2 else 0
                                   for i in range(n))
        self.rule = [0, 1, 0, 1, 1, 0, 1, 0]
    # ---
        
    @property
    def state(self):
        return [ '#' if cell else '.' for cell in self.gridstate ]
    # ---
        
    def process(self, time):
        'Move one time step forward.'
        self.gridstate = self.applyrule(self.gridstate)
    # ---
        
    def applyrule(self, grid):
        'Apply automaton rule to the given grid.'
        # Work on a copy
        newgrid = list(grid)
        
        for i,cell in enumerate(grid):
            # Cells at the edges are constant
            if 1 <= i < len(grid)-1:
                neighbors = grid[i-1:i+2]
                newgrid[i] = self.rule[self.process_neighbor(neighbors)]
        # Return an immutable new grid
        return tuple(newgrid)
                
    def process_neighbor(self, neighbors):
        'Convert neighborhood to integer to apply rule.'
        a, b, c = neighbors
        return c + b*2 + a*4
    
    def __str__(self):
        return ''.join(self.state)
# --- CellularAutomaton


def simplelogging(entity):
    print(entity)
# ---

In [137]:
# Intended use case...
class CellularAutomaton(Entity):
    'A simple cellular automaton.'
    
    def __init__(self, n=25):
        # Place a single active cell in the middle
        self.gridstate = tuple(1 if i == n//2 else 0
                                   for i in range(n))
        self.rule = [0, 1, 0, 1, 1, 0, 1, 0]
    # ---
        
    @property
    def state(self):
        return [ '#' if cell else '.' for cell in self.gridstate ]
    # ---
        
    def process(self, time):
        'Move one time step forward.'
        self.gridstate = self.applyrule(self.gridstate)
    # ---
        
    def applyrule(self, grid):
        'Apply automaton rule to the given grid.'
        # Work on a copy
        newgrid = list(grid)
        
        for i,cell in enumerate(grid):
            # Cells at the edges are constant
            if 1 <= i < len(grid)-1:
                neighbors = grid[i-1:i+2]
                newgrid[i] = self.rule[self.process_neighbor(neighbors)]
        # Return an immutable new grid
        return tuple(newgrid)
                
    def process_neighbor(self, neighbors):
        'Convert neighborhood to integer to apply rule.'
        a, b, c = neighbors
        return c + b*2 + a*4
    
    def __str__(self):
        return ''.join(self.state)
# --- CellularAutomaton



def simplelogging(entity):
    print(entity)
# ---


## < --- --- ----


sys = System()

sys.add(CellularAutomaton(100), 
        name='Sierpinsky')

sys.run(steps=50, 
        prehooks={simplelogging:['Sierpinsky']})

print('Final state: ', sys['Sierpinsky'])
print(sys.stateof('Sierpinsky'))

..................................................#.................................................
.................................................#.#................................................
................................................#...#...............................................
...............................................#.#.#.#..............................................
..............................................#.......#.............................................
.............................................#.#.....#.#............................................
............................................#...#...#...#...........................................
...........................................#.#.#.#.#.#.#.#..........................................
..........................................#...............#.........................................
.........................................#.#.............#.#...............................

In [138]:
sys

In [139]:
sys.entities

{<__main__.CellularAutomaton at 0x7f28e43a20b8>}

In [140]:
for e in sys.entities:
    help(e)

Help on CellularAutomaton in module __main__ object:

class CellularAutomaton(Entity)
 |  A simple cellular automaton.
 |  
 |  Method resolution order:
 |      CellularAutomaton
 |      Entity
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, n=25)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  applyrule(self, grid)
 |      Apply automaton rule to the given grid.
 |  
 |  process(self, time)
 |      Move one time step forward.
 |  
 |  process_neighbor(self, neighbors)
 |      Convert neighborhood to integer to apply rule.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  state
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Entity:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      lis

We see now that several changes have been done to the code. The hooks where added with three flavors: inithooks (executed before the simulation starts), prehooks (executed before any step) and hooks (executed at the end of every step). This developed in a major factorization of how the interactions are applied, as the hooks are implemented using the same mechanism.